## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [7]:
project_path_s = [path.curdir, "TrustZone-S"]
project_path_s

['.', 'TrustZone-S']

In [8]:
project_path_ns = [path.curdir, "TrustZone-NS"]
project_path_ns

['.', 'TrustZone-NS']

In [9]:
atprogram(path.abspath(path.join(*project_path_ns)), verbose=2)
atprogram(path.abspath(path.join(*project_path_s)), verbose=2, erase=False)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:41415
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00008000. Size = 0x000006d4.
[DEBUG] Memory segment base written at 0x000086d4. Size = 0x00000064.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "program"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:41420
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Memory segment base written a

0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [13]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_trustzone"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment TrustZone",
}

Stop criteria to pass to the logger:

In [11]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [13]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger.log(400,stop_fn)
    data = dgilib.data

In [14]:
print(data)

Interfaces:
	 256:   power,   samples: 3551000
	  48:   gpio,    samples:   32644



# Store Data

In [ ]:
import pickle
pickle.dump(data, open("trustzone_logger_data.p", "wb"))

# Load Data

In [ ]:
data = pickle.load(open("trustzone_logger_data.p", "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [15]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [16]:
nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [17]:
nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [18]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(nsc_store_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(nsc_store_charge) - 1}")

MAX_NUM_BYTES: 4080


In [19]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [20]:
results = []
for y in [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 4080
    # variables        = 2
    chi-square         = 6.0008e-13
    reduced chi-square = 1.4715e-16
    Akaike info crit   = -148734.738
    Bayesian info crit = -148722.111
[[Variables]]
    slope:      8.6781e-10 +/- 1.6124e-13 (0.02%) (init = 0)
    intercept:  2.2524e-08 +/- 3.7989e-10 (1.69%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.866

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 4080
    # variables        = 2
    chi-square         = 5.7582e-13
    reduced chi-square = 1.4120e-16
    Akaike info crit   = -148903.059
    Bayesian info crit = -148890.431
[[Variables]]
    slope:      8.7152e-10 +/- 1.5795e-13 (0.02%) (init = 0)
    intercept:  1.8287e-08 +/- 3.7214e-10 (2.03%) (init = 1)
[[Correlations]] (unrep

In [21]:
fig2 = plt.figure(figsize=(9, 8))
fig2.canvas.set_window_title("Analysis TrustZone")

In [22]:
charge_color = 'r'
time_color = 'b'

In [23]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [24]:
lines = []
lines += ax1.plot(num_bytes, nsc_store_charge, charge_color+'-', label='TrustZone Store Charge')
lines += ax1.plot(num_bytes, nsc_load_charge, charge_color+'--', label='TrustZone Load Charge')
lines += ax2.plot(num_bytes, nsc_store_time, time_color+'-', label='TrustZone Store Time')
lines += ax2.plot(num_bytes, nsc_load_time, time_color+'--', label='TrustZone Load Time')
ax1.legend(handles=lines)
# [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
ax1.set_title(f"TrustZone Store Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Load Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Store Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"TrustZone Load Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [25]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(nsc_store_charge, open("trustzone_store_charge.p", "wb"))
pickle.dump(nsc_load_charge, open("trustzone_load_charge.p", "wb"))
pickle.dump(nsc_store_time, open("trustzone_store_time.p", "wb"))
pickle.dump(nsc_load_time, open("trustzone_load_time.p", "wb"))

## Write config file

In [17]:
import json

config = {}
config["name"] = "TrustZone"
config["project_paths"] = [project_path_ns, project_path_s]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["TrustZone Write"], 3: ["TrustZone Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["TrustZone Write"],
                                        "load": ["TrustZone Read"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": 1}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)